# The Cancer Gene Atlas Cancer (TCGA) - Bladder Urothelial Carcinoma (BLCA)

Author: Moshe Silverstein <br/>
Date: 7-17 <br/>
Data Source: https://cancergenome.nih.gov/

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import untility_functions as uf
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

In [2]:
import seaborn as sns
sns.set(color_codes=True)
np.random.seed(sum(map(ord, "distributions")))

In [3]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [4]:
importlib.reload(uf)

<module 'untility_functions' from '/Users/moshesilverstein/Documents/Harmonizome/TCGA/untility_functions.py'>

# Versions Of Modules In Use

In [5]:
%load_ext version_information
%version_information numpy, pandas, clustergrammer_widget 

Software versions
Python 3.5.2 64bit [GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
IPython 5.3.0
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.13.1
pandas 0.21.0
clustergrammer_widget 1.9.0
Mon Apr 23 12:35:35 2018 EDT

# Functions

In [6]:
def getGenes(inputDF):
    
    inputDF.reset_index(inplace=True)
    
    inputDF.rename(columns={'index':'Gene Symbol'}, inplace=True)

    inputDF.set_index('Gene Symbol', inplace=True)

    lst = []

    for index in  inputDF.index:
        lst.append(index.split('|')[5])

    inputDF.index = lst

In [7]:
def getSampleID(inputDF, metaDF):
    
    lst = []
    
    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()

        ID = ('-').join(metaDF.loc[col, 'a_AliquotBarcode'].split('-')[0:4])

        lst.append(ID)

        
    inputDF.columns = lst

In [8]:
def getSampleData(inputDF, metaDF):
    
    lst = []

    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()
        
#         ID = ('-').join(meta.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        ID = ('-').join(col.split('-')[0:3])
        
        donor = 'Donor:'+ID
        
        if str(col.split('-')[3]) == '01A':
            sampleType = 'Primary Solid Tumor'
        elif str(col.split('-')[3]) == '11A':
            sampleType= 'Solid Tissue Normal'
        
        if ID in metaDF.index:
            info = (donor, 
                'Age:'+str(metaDF.loc[ID, 'age_at_initial_pathologic_diagnosis']),
                'Gender:'+str(metaDF.loc[ID, 'gender']),    
                'SampleType:'+sampleType,
                'Stage:'+str(metaDF.loc[ID, 'pathologic_stage']),
                'Histology:'+str(metaDF.loc[ID, 'histological_type']),
                'Tissue:'+str(metaDF.loc[ID, 'tumor_tissue_site']), 
                'Tumor:'+str(metaDF.loc[ID, 'tumor_type']),    
                'Grade'+str(metaDF.loc[ID, 'neoplasm_histologic_grade'])
#                 'SampleType:'+str(ID.split('-')[3])
               )
#             for col in metaDF.columns:
#                 if len(metaDF[col].dropna()) >= .5*len(metaDF[col]) and len(metaDF[col].unique()) > 2 and len(metaDF[col].unique()) < len(metaDF[col]) and 'age' not in col:
#                     info = info + (str(col)+':'+str(metaDF.loc[ID, col]),)
        lst.append(info)

        
    inputDF.columns = lst

# Path to Output Files

In [9]:
path = '/Users/moshesilverstein/Documents/Harmonizome/TCGA/Output/'

# Load Meta Data

In [10]:
meta = pd.read_csv('Input/TCGA_Metadata.csv', index_col=9)

In [11]:
meta.head()

,a_AliquotBarcode,b_disease,a_objectSizeBytes,b_uploaded,b_state,b_library_type,b_center,b_assembly,a_GCSobject
a_CGHubAnalysisID,,,,,,,,,
3a8e6a74-137f-468f-8987-fa0acdde2836,TCGA-CS-6188-01A-11R-1896-07,LGG,6354301047,8/14/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
c63d44f1-c9ad-4391-b1aa-0feb3713441b,TCGA-EO-A22Y-01A-11R-A180-07,UCEC,5497637066,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
21912089-1e42-4bcc-9ad9-fe9a9b88fb09,TCGA-94-A5I4-01A-11R-A26W-07,LUSC,4600317850,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
6b86e544-0372-434a-9f28-6fa4075dd228,TCGA-N5-A4RV-01A-21R-A28V-07,UCS,4964387768,8/30/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
30518eb4-2783-4988-bcd4-36a1a8bb4dfa,TCGA-BK-A56F-01A-32R-A27V-07,UCEC,5668623460,8/31/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...


In [12]:
meta.shape

(11373, 9)

# Load Sample MetaData

In [13]:
sample_meta = pd.read_csv('Input/metadata_Bladder Urothelial Carcinoma [BLCA]', sep='\t', index_col=1)

In [14]:
sample_meta.head()

,Unnamed: 0,Study,Project,ParticipantUUID,TSSCode,age_at_initial_pathologic_diagnosis,anatomic_neoplasm_subdivision,batch_number,bcr,clinical_M,...,BMI,age_began_smoking_in_years,h_pylori_infection,other_dx,other_malignancy_anatomic_site,other_malignancy_histological_type,other_malignancy_malignancy_type,stopped_smoking_year,venous_invasion,year_of_tobacco_smoking_onset
ParticipantBarcode,,,,,,,,,,,,,,,,,,,,,
TCGA-4Z-AA7R,8946,BLCA,TCGA,72FE54B5-C1C8-468A-954A-09992429512A,4Z,73.0,Bladder - NOS,401,Nationwide Children's Hospital,NaN,...,25.0,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-4Z-AA89,8947,BLCA,TCGA,604C0D60-6BA9-4D2F-98DB-E310C58BC8CA,4Z,60.0,Bladder - NOS,401,Nationwide Children's Hospital,NaN,...,31.4,NaN,NaN,"Yes, History of Prior Malignancy",Bladder,"Other, specify",Prior Malignancy,NaN,NaN,NaN
TCGA-BT-A20U,8948,BLCA,TCGA,83b41d35-69b1-4218-be1a-8821d8a6899c,BT,70.0,Bladder - NOS,128,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-BT-A20P,8949,BLCA,TCGA,f48937ed-e294-41f0-8872-7010391167a9,BT,81.0,NaN,128,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-BT-A20Q,8950,BLCA,TCGA,419deaac-ea45-4bdd-9fa0-b5cd8429b44f,BT,73.0,NaN,128,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
sample_meta.shape

(412, 70)

# Load Data

In [16]:
matrix = pd.read_csv('Input/TCGA_BLCA_tpm.tsv', sep='\t', index_col=0)

In [17]:
matrix.head()

,1be40cf0-10c9-4cb2-87b8-1dcab5120d3a,f6c55abd-c73c-4077-a3fe-57f26fb3b076,d0948992-019a-4462-8f4a-dfb75d6f4dba,23ed1ead-2749-43b4-bb18-ae8746e63ca0,27e1044b-c193-4747-97ea-fb3a9bebfdfe,a4768fe0-4634-44b1-8a2e-6f10939b994a,1f17d968-d235-4254-8ff8-0d9dcfa659a7,8b12eac0-dd9b-4c81-97c4-6320b5b7db15,b5e83b18-8d86-4ad0-8cb6-6611773f5fa2,f22342e9-0425-4ea2-ac8b-6ca6f6253319,...,f6fc1b19-8ded-479a-ac68-a08b92ced591,2bda95ae-050f-4e14-919b-09626b119581,14807042-bd63-49c7-a1a3-0c1eeb4eddba,4acd03a0-c319-43d7-9b3e-5e2a28d0bb78,2a00d299-bb47-41e5-b9f5-5f313dddabb5,3e86531f-ab27-4293-9e8e-950baedf480b,662090c7-526a-4382-84f2-43709274fcb6,9ed08d6d-2e58-4694-8941-adf02a377798,54f0ffc2-c781-415c-b8b6-36fa5f5b1c7f,6815303c-dfb4-45c5-98db-84d29b3a8306
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,206.703000,281.951000,196.99300,181.768000,212.70100,226.8390,141.012000,156.647000,295.643000,249.642000,...,105.2520,250.3540,202.670000,306.4590,1.152640e+02,313.6020,275.993000,255.039000,176.573000,154.618000
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,41.421900,21.685300,50.85610,38.956200,7.68129,34.9029,22.188000,45.407200,64.117700,78.022200,...,29.7718,0.0000,45.399200,55.1970,0.000000e+00,9.3533,50.934400,10.052800,23.899100,12.325400
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,23.121600,22.730000,16.48340,14.761300,23.11740,24.1605,19.148300,47.781600,28.526100,33.920200,...,27.5218,18.5071,21.105300,37.6959,2.905120e+01,18.1939,10.882500,20.058800,25.374400,10.216400
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,58.099900,98.385000,68.87530,48.183800,107.97700,78.9584,23.385700,73.416300,42.253700,51.326400,...,42.9364,44.8478,106.308000,76.7228,6.490240e+01,75.6593,37.154700,49.885700,71.769400,22.230300
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.101862,0.256019,3.24345,0.985131,0.00000,0.0000,0.223034,0.032648,0.117981,0.294343,...,0.0000,0.0000,0.044062,0.0000,1.648510e-08,0.0000,0.513749,0.154934,0.490759,0.302087


In [18]:
matrix.shape

(199169, 433)

# Get Only Protein Coding Genes

In [19]:
lst = []

for i,index in enumerate(matrix.index):
    
    progressPercent = ((i+1)/len(matrix.index)*100)

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(matrix.index)))
    sys.stdout.flush()
                                 
    lst.append(index.split('|')[7])    
                       
matrix['Gene Type'] = lst

matrix = matrix[matrix['Gene Type'] == 'protein_coding']

matrix.drop('Gene Type', axis=1, inplace=True)

In [20]:
matrix.head()

,1be40cf0-10c9-4cb2-87b8-1dcab5120d3a,f6c55abd-c73c-4077-a3fe-57f26fb3b076,d0948992-019a-4462-8f4a-dfb75d6f4dba,23ed1ead-2749-43b4-bb18-ae8746e63ca0,27e1044b-c193-4747-97ea-fb3a9bebfdfe,a4768fe0-4634-44b1-8a2e-6f10939b994a,1f17d968-d235-4254-8ff8-0d9dcfa659a7,8b12eac0-dd9b-4c81-97c4-6320b5b7db15,b5e83b18-8d86-4ad0-8cb6-6611773f5fa2,f22342e9-0425-4ea2-ac8b-6ca6f6253319,...,f6fc1b19-8ded-479a-ac68-a08b92ced591,2bda95ae-050f-4e14-919b-09626b119581,14807042-bd63-49c7-a1a3-0c1eeb4eddba,4acd03a0-c319-43d7-9b3e-5e2a28d0bb78,2a00d299-bb47-41e5-b9f5-5f313dddabb5,3e86531f-ab27-4293-9e8e-950baedf480b,662090c7-526a-4382-84f2-43709274fcb6,9ed08d6d-2e58-4694-8941-adf02a377798,54f0ffc2-c781-415c-b8b6-36fa5f5b1c7f,6815303c-dfb4-45c5-98db-84d29b3a8306
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,206.703000,281.951000,196.99300,181.768000,212.70100,226.8390,141.012000,156.647000,295.643000,249.642000,...,105.2520,250.3540,202.670000,306.4590,1.152640e+02,313.6020,275.993000,255.039000,176.573000,154.618000
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,41.421900,21.685300,50.85610,38.956200,7.68129,34.9029,22.188000,45.407200,64.117700,78.022200,...,29.7718,0.0000,45.399200,55.1970,0.000000e+00,9.3533,50.934400,10.052800,23.899100,12.325400
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,23.121600,22.730000,16.48340,14.761300,23.11740,24.1605,19.148300,47.781600,28.526100,33.920200,...,27.5218,18.5071,21.105300,37.6959,2.905120e+01,18.1939,10.882500,20.058800,25.374400,10.216400
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,58.099900,98.385000,68.87530,48.183800,107.97700,78.9584,23.385700,73.416300,42.253700,51.326400,...,42.9364,44.8478,106.308000,76.7228,6.490240e+01,75.6593,37.154700,49.885700,71.769400,22.230300
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.101862,0.256019,3.24345,0.985131,0.00000,0.0000,0.223034,0.032648,0.117981,0.294343,...,0.0000,0.0000,0.044062,0.0000,1.648510e-08,0.0000,0.513749,0.154934,0.490759,0.302087


In [21]:
matrix.shape

(79930, 433)

# Map Sample Meta Data to Sample ID

In [22]:
getSampleID(matrix, meta)

In [23]:
matrix.head()

,TCGA-FD-A5C0-01A,TCGA-FD-A5C1-01A,TCGA-K4-A5RI-01A,TCGA-FD-A5BZ-01A,TCGA-CF-A5UA-01A,TCGA-E7-A5KE-01A,TCGA-K4-A5RI-11A,TCGA-E7-A5KF-01A,TCGA-FD-A5BY-01A,TCGA-HQ-A5NE-01A,...,TCGA-ZF-A9R5-01A,TCGA-4Z-AA89-01A,TCGA-DK-AA6U-01A,TCGA-4Z-AA80-01A,TCGA-4Z-AA87-01A,TCGA-G2-AA3B-01A,TCGA-XF-A9SI-01A,TCGA-ZF-AA52-01A,TCGA-XF-A9T0-01A,TCGA-XF-AAN8-01A
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,206.703000,281.951000,196.99300,181.768000,212.70100,226.8390,141.012000,156.647000,295.643000,249.642000,...,105.2520,250.3540,202.670000,306.4590,1.152640e+02,313.6020,275.993000,255.039000,176.573000,154.618000
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,41.421900,21.685300,50.85610,38.956200,7.68129,34.9029,22.188000,45.407200,64.117700,78.022200,...,29.7718,0.0000,45.399200,55.1970,0.000000e+00,9.3533,50.934400,10.052800,23.899100,12.325400
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,23.121600,22.730000,16.48340,14.761300,23.11740,24.1605,19.148300,47.781600,28.526100,33.920200,...,27.5218,18.5071,21.105300,37.6959,2.905120e+01,18.1939,10.882500,20.058800,25.374400,10.216400
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,58.099900,98.385000,68.87530,48.183800,107.97700,78.9584,23.385700,73.416300,42.253700,51.326400,...,42.9364,44.8478,106.308000,76.7228,6.490240e+01,75.6593,37.154700,49.885700,71.769400,22.230300
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.101862,0.256019,3.24345,0.985131,0.00000,0.0000,0.223034,0.032648,0.117981,0.294343,...,0.0000,0.0000,0.044062,0.0000,1.648510e-08,0.0000,0.513749,0.154934,0.490759,0.302087


# Map Gene Symbol to Gene ID

In [24]:
getGenes(matrix)

In [25]:
matrix.head()

,TCGA-FD-A5C0-01A,TCGA-FD-A5C1-01A,TCGA-K4-A5RI-01A,TCGA-FD-A5BZ-01A,TCGA-CF-A5UA-01A,TCGA-E7-A5KE-01A,TCGA-K4-A5RI-11A,TCGA-E7-A5KF-01A,TCGA-FD-A5BY-01A,TCGA-HQ-A5NE-01A,...,TCGA-ZF-A9R5-01A,TCGA-4Z-AA89-01A,TCGA-DK-AA6U-01A,TCGA-4Z-AA80-01A,TCGA-4Z-AA87-01A,TCGA-G2-AA3B-01A,TCGA-XF-A9SI-01A,TCGA-ZF-AA52-01A,TCGA-XF-A9T0-01A,TCGA-XF-AAN8-01A
ARF5,206.703000,281.951000,196.99300,181.768000,212.70100,226.8390,141.012000,156.647000,295.643000,249.642000,...,105.2520,250.3540,202.670000,306.4590,1.152640e+02,313.6020,275.993000,255.039000,176.573000,154.618000
M6PR,41.421900,21.685300,50.85610,38.956200,7.68129,34.9029,22.188000,45.407200,64.117700,78.022200,...,29.7718,0.0000,45.399200,55.1970,0.000000e+00,9.3533,50.934400,10.052800,23.899100,12.325400
ESRRA,23.121600,22.730000,16.48340,14.761300,23.11740,24.1605,19.148300,47.781600,28.526100,33.920200,...,27.5218,18.5071,21.105300,37.6959,2.905120e+01,18.1939,10.882500,20.058800,25.374400,10.216400
FKBP4,58.099900,98.385000,68.87530,48.183800,107.97700,78.9584,23.385700,73.416300,42.253700,51.326400,...,42.9364,44.8478,106.308000,76.7228,6.490240e+01,75.6593,37.154700,49.885700,71.769400,22.230300
CYP26B1,0.101862,0.256019,3.24345,0.985131,0.00000,0.0000,0.223034,0.032648,0.117981,0.294343,...,0.0000,0.0000,0.044062,0.0000,1.648510e-08,0.0000,0.513749,0.154934,0.490759,0.302087


# Histogram of First Sample

In [26]:
# matrix.iloc[:, 0].hist(bins=100, log=True);
sns.distplot(matrix.iloc[:, 0]);

# Histogram of First Gene

In [27]:
# matrix.iloc[0, :].hist(bins=100);
sns.distplot(matrix.iloc[0, :]);

# Save Unfiltered Matrix to File

In [28]:
filename = path+'tcga_blca_matrix_unfilltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
matrix.to_csv(filename, sep='\t', compression='gzip')

# Filtered

In [38]:
normalized_matrix = matrix.copy()

In [39]:
normalized_matrix = normalized_matrix.groupby(level=0).max()

# Remove any data with more than 95% missing and imput remaining mssing values to matrix mean

In [40]:
normalized_matrix = uf.removeAndImpute(normalized_matrix)

In [41]:
normalized_matrix.head()

,TCGA-FD-A5C0-01A,TCGA-FD-A5C1-01A,TCGA-K4-A5RI-01A,TCGA-FD-A5BZ-01A,TCGA-CF-A5UA-01A,TCGA-E7-A5KE-01A,TCGA-K4-A5RI-11A,TCGA-E7-A5KF-01A,TCGA-FD-A5BY-01A,TCGA-HQ-A5NE-01A,...,TCGA-ZF-A9R5-01A,TCGA-4Z-AA89-01A,TCGA-DK-AA6U-01A,TCGA-4Z-AA80-01A,TCGA-4Z-AA87-01A,TCGA-G2-AA3B-01A,TCGA-XF-A9SI-01A,TCGA-ZF-AA52-01A,TCGA-XF-A9T0-01A,TCGA-XF-AAN8-01A
A1BG,0.206866,0.206866,0.206866,0.206866,0.206866,0.206866,0.206866,0.206866,0.206866,0.206866,...,0.206866,0.206866,0.206866,0.206866,0.046470,0.206866,0.206866,0.206866,0.206866,0.206866
A1CF,0.011653,0.009265,0.004921,0.019710,0.104465,0.015587,0.002622,0.019706,0.005600,0.009723,...,0.032608,0.017529,0.035784,3.593840,0.104465,4.501000,0.015567,0.014680,0.012077,0.104465
A2M,144.389000,63.858900,86.077700,149.723000,43.623200,30.748300,786.077000,28.371100,71.561600,103.862000,...,35.538300,10.575000,40.496600,26.231900,17.199700,9.737620,136.889000,307.219000,31.512900,253.903000
A2ML1,79.163600,0.440967,0.764300,0.128850,0.094627,0.852957,0.110424,8.466760,8.359120,53.034800,...,0.271362,0.696200,14.399400,0.796607,3.224050,5.121920,27.041600,23.122500,0.153596,0.061397
A3GALT2,0.060978,0.060978,0.060978,0.159453,0.060978,0.045954,0.060978,0.020820,0.060978,0.060978,...,0.060978,0.060978,0.060978,0.060978,0.189970,0.027027,0.060978,0.060978,0.060978,0.060978


In [42]:
normalized_matrix.shape

(19060, 433)

# logCPM Transform

Raw counts were normalized to log10-Counts Per Million (logCPM) by dividing each column by the total sum of its counts, multiplying it by 106, followed by the application of a log10-transform.

In [43]:
samples = normalized_matrix.columns.tolist()

In [44]:
normalized_matrix.columns = np.arange(0, len(normalized_matrix.columns))

In [45]:
for col in normalized_matrix.columns:

    total = normalized_matrix[col].sum()
    
    normalized_matrix[col] = normalized_matrix[col].apply(lambda x: np.log10(((x)/total)*10**6))

In [46]:
normalized_matrix.columns = samples

In [47]:
normalized_matrix.head()

,TCGA-FD-A5C0-01A,TCGA-FD-A5C1-01A,TCGA-K4-A5RI-01A,TCGA-FD-A5BZ-01A,TCGA-CF-A5UA-01A,TCGA-E7-A5KE-01A,TCGA-K4-A5RI-11A,TCGA-E7-A5KF-01A,TCGA-FD-A5BY-01A,TCGA-HQ-A5NE-01A,...,TCGA-ZF-A9R5-01A,TCGA-4Z-AA89-01A,TCGA-DK-AA6U-01A,TCGA-4Z-AA80-01A,TCGA-4Z-AA87-01A,TCGA-G2-AA3B-01A,TCGA-XF-A9SI-01A,TCGA-ZF-AA52-01A,TCGA-XF-A9T0-01A,TCGA-XF-AAN8-01A
A1BG,-0.521334,-0.496026,-0.507455,-0.479326,-0.484719,-0.487731,-0.463018,-0.483162,-0.504730,-0.510394,...,-0.463649,-0.414163,-0.497923,-0.486639,-1.062854,-0.487488,-0.445638,-0.491119,-0.449509,-0.491924
A1CF,-1.770604,-1.844855,-2.131103,-1.500317,-0.781436,-1.610668,-2.360111,-1.504241,-2.072230,-1.838262,...,-1.266007,-1.486085,-1.259928,0.753231,-0.711052,0.850132,-1.569131,-1.640074,-1.683254,-0.788641
A2M,2.322511,1.993506,2.111747,2.380274,1.839310,1.684401,3.116758,1.654025,2.034261,2.190373,...,1.771358,1.294428,1.793807,1.616502,1.505497,1.185275,2.375041,2.680640,1.733290,2.597055
A2ML1,2.061503,-0.167309,0.060120,-0.684931,-0.824394,0.127507,-0.735644,1.128866,1.101742,1.898477,...,-0.345789,0.112882,1.344733,0.098916,0.778378,0.906255,1.670705,1.557226,-0.578818,-1.019466
A3GALT2,-1.051853,-1.026545,-1.037974,-0.592382,-1.015238,-1.141096,-0.993537,-1.480376,-1.035249,-1.040913,...,-0.994168,-0.944682,-1.028441,-1.017158,-0.451339,-1.371384,-0.976157,-1.021638,-0.980028,-1.022443


# Normalize Matrix (Quantile Normalize the matrix for the columns)

In [48]:
normalized_matrix = uf.quantileNormalize(normalized_matrix)

In [49]:
normalized_matrix.head()

,TCGA-FD-A5C0-01A,TCGA-FD-A5C1-01A,TCGA-K4-A5RI-01A,TCGA-FD-A5BZ-01A,TCGA-CF-A5UA-01A,TCGA-E7-A5KE-01A,TCGA-K4-A5RI-11A,TCGA-E7-A5KF-01A,TCGA-FD-A5BY-01A,TCGA-HQ-A5NE-01A,...,TCGA-ZF-A9R5-01A,TCGA-4Z-AA89-01A,TCGA-DK-AA6U-01A,TCGA-4Z-AA80-01A,TCGA-4Z-AA87-01A,TCGA-G2-AA3B-01A,TCGA-XF-A9SI-01A,TCGA-ZF-AA52-01A,TCGA-XF-A9T0-01A,TCGA-XF-AAN8-01A
A1BG,-0.519384,-0.445212,-0.474007,-0.645551,-0.313940,-0.446403,-0.567908,-0.405965,-0.516564,-0.568395,...,-0.453814,-0.598606,-0.487635,-0.479056,-1.299186,-0.318591,-0.660006,-0.510582,-0.450701,-0.511070
A1CF,-1.788018,-1.758165,-2.072907,-1.652480,-0.654353,-1.601245,-2.660441,-1.462867,-2.092530,-1.910028,...,-1.268424,-1.899506,-1.262040,0.659299,-0.885019,0.986438,-1.728125,-1.657348,-1.695916,-0.822710
A2M,2.220313,1.944709,2.067362,2.302915,1.872326,1.665868,3.050009,1.596313,1.913866,2.063399,...,1.854893,1.264340,1.725494,1.472689,1.518744,1.303225,2.313000,2.663036,1.716403,2.564918
A2ML1,1.967202,-0.108422,0.075431,-0.827234,-0.698940,0.154603,-0.814358,1.068199,1.000303,1.766609,...,-0.321268,0.051236,1.249840,0.105119,0.841889,1.041555,1.591387,1.542549,-0.584216,-1.061424
A3GALT2,-1.045357,-0.979433,-1.002958,-0.746488,-0.911489,-1.124670,-1.059937,-1.439730,-1.036784,-1.088376,...,-1.000101,-1.241858,-1.013368,-1.045648,-0.572887,-1.200342,-1.127974,-1.044777,-1.002958,-1.064614


# Normalize Matrix (z-score the rows)

In [50]:
uf.zscore(normalized_matrix, 'row')

In [51]:
normalized_matrix.head()

,TCGA-FD-A5C0-01A,TCGA-FD-A5C1-01A,TCGA-K4-A5RI-01A,TCGA-FD-A5BZ-01A,TCGA-CF-A5UA-01A,TCGA-E7-A5KE-01A,TCGA-K4-A5RI-11A,TCGA-E7-A5KF-01A,TCGA-FD-A5BY-01A,TCGA-HQ-A5NE-01A,...,TCGA-ZF-A9R5-01A,TCGA-4Z-AA89-01A,TCGA-DK-AA6U-01A,TCGA-4Z-AA80-01A,TCGA-4Z-AA87-01A,TCGA-G2-AA3B-01A,TCGA-XF-A9SI-01A,TCGA-ZF-AA52-01A,TCGA-XF-A9T0-01A,TCGA-XF-AAN8-01A
A1BG,-0.256672,0.376096,0.130442,-1.333012,1.495987,0.365935,-0.670629,0.710914,-0.232612,-0.674782,...,0.302715,-0.932518,0.014181,0.087369,-6.909217,1.456308,-1.456326,-0.181582,0.329267,-0.185739
A1CF,-0.685126,-0.615402,-1.350515,-0.368564,1.962666,-0.248898,-2.722761,0.074297,-1.396346,-0.970092,...,0.528439,-0.945518,0.543349,5.030837,1.423922,5.794906,-0.545240,-0.379933,-0.470014,1.569451
A2M,0.503475,-0.114831,0.160336,0.688789,-0.277219,-0.740396,2.364858,-0.896441,-0.184024,0.151446,...,-0.316329,-1.641206,-0.606629,-1.173785,-1.070462,-1.553971,0.711413,1.496703,-0.627024,1.276581
A2ML1,1.339511,-0.487181,-0.325378,-1.119785,-1.006878,-0.255701,-1.108453,0.548327,0.488573,1.162975,...,-0.674500,-0.346671,0.708183,-0.299250,0.349158,0.524878,1.008768,0.965787,-0.905912,-1.325889
A3GALT2,-0.080199,0.349913,0.196423,1.869719,0.793200,-0.597665,-0.175324,-2.653214,-0.024262,-0.360870,...,0.215068,-1.362237,0.128511,-0.082100,3.002350,-1.091367,-0.619215,-0.076417,0.196423,-0.205835


# Merge Like Column (by taking the mean)

In [52]:
normalized_matrix = uf.merge(normalized_matrix, 'column', 'mean')

In [53]:
normalized_matrix.shape

(19060, 430)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [54]:
uf.mapgenesymbols(normalized_matrix)

In [55]:
normalized_matrix.shape

(18638, 430)

# Merge Duplicate Genes By Rows

In [56]:
normalized_matrix = uf.merge(normalized_matrix, 'row', 'mean')

In [57]:
normalized_matrix.shape

(18549, 430)

# Histogram of First Sample

In [58]:
# normalized_matrix.iloc[:, 0].hist(bins=100, log=True);
sns.distplot(normalized_matrix.iloc[:, 10]);

# Histogram of First Gene

In [59]:
# normalized_matrix.iloc[1, :].hist(bins=50);
sns.distplot(normalized_matrix.iloc[10, :]);

# Save Filtered Matrix

In [60]:
filename = path+'tcga_blca_matrix_filltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
normalized_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [61]:
gene_list = uf.createGeneList(normalized_matrix)

In [62]:
gene_list.head()

,GeneSym,GeneID
0,A1BG,1
1,A1CF,29974
2,A2M,2
3,A2ML1,144568
4,A3GALT2,127550


In [63]:
gene_list.shape

(18549, 2)

# Save Gene List

In [64]:
filename = path+'tcga_blca_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute List

In [65]:
metaData = pd.DataFrame(index=normalized_matrix.columns)

for index in metaData.index:
    metaData.loc[index, 'Age'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'age_at_initial_pathologic_diagnosis']
    metaData.loc[index, 'Gender'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'gender']
    metaData.loc[index, 'Pathologic M'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_M']
    metaData.loc[index, 'Pathologic N'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_N']
    metaData.loc[index, 'Pathologic T'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_T']
    metaData.loc[index, 'Pathologic Stage'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_stage']
    metaData.loc[index, 'Histological Type'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'histological_type']
    if str(index.split('-')[3]) == '01A':
        sampleType = 'Primary Solid Tumor'
    elif str(index.split('-')[3]) == '11A':
        sampleType= 'Solid Tissue Normal'
    metaData.loc[index, 'Sample Type'] = sampleType
    metaData.loc[index, 'Tumor Tissue Site'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'tumor_tissue_site']
    metaData.loc[index, 'Neoplasm Histologic Grade'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'neoplasm_histologic_grade']
    

In [66]:
attribute_list = uf.createAttributeList(normalized_matrix, metaData)

In [67]:
attribute_list.head()

,Age,Gender,Pathologic M,Pathologic N,Pathologic T,Pathologic Stage,Histological Type,Sample Type,Tumor Tissue Site,Neoplasm Histologic Grade
Attributes,,,,,,,,,,
TCGA-2F-A9KO-01A,63,MALE,M0,N1,T3,Stage IV,Muscle invasive urothelial carcinoma (pT2 or a...,Primary Solid Tumor,Bladder,High Grade
TCGA-2F-A9KP-01A,66,MALE,MX,N2,T3a,Stage IV,Muscle invasive urothelial carcinoma (pT2 or a...,Primary Solid Tumor,Bladder,High Grade
TCGA-2F-A9KQ-01A,69,MALE,M0,N0,T3a,Stage III,Muscle invasive urothelial carcinoma (pT2 or a...,Primary Solid Tumor,Bladder,High Grade
TCGA-2F-A9KR-01A,59,FEMALE,M0,N0,T3a,Stage III,Muscle invasive urothelial carcinoma (pT2 or a...,Primary Solid Tumor,Bladder,High Grade
TCGA-2F-A9KT-01A,83,MALE,M0,N0,T2b,Stage II,Muscle invasive urothelial carcinoma (pT2 or a...,Primary Solid Tumor,Bladder,High Grade


In [68]:
attribute_list.shape

(430, 10)

# Save Attribute List

In [69]:
filename = path+'tcga_blca_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', compression='gzip')

# Create matrix of Standardized values (values between -1, and 1)

In [70]:
standard_matrix = uf.createStandardizedMatrix(normalized_matrix)

In [71]:
standard_matrix.head()

,TCGA-2F-A9KO-01A,TCGA-2F-A9KP-01A,TCGA-2F-A9KQ-01A,TCGA-2F-A9KR-01A,TCGA-2F-A9KT-01A,TCGA-2F-A9KW-01A,TCGA-4Z-AA7M-01A,TCGA-4Z-AA7N-01A,TCGA-4Z-AA7O-01A,TCGA-4Z-AA7Q-01A,...,TCGA-ZF-AA4X-01A,TCGA-ZF-AA51-01A,TCGA-ZF-AA52-01A,TCGA-ZF-AA53-01A,TCGA-ZF-AA54-01A,TCGA-ZF-AA56-01A,TCGA-ZF-AA58-01A,TCGA-ZF-AA5H-01A,TCGA-ZF-AA5N-01A,TCGA-ZF-AA5P-01A
,,,,,,,,,,,,,,,,,,,,,
A1BG,-0.474030,0.269926,0.488587,-0.618123,-0.018384,-0.274142,-0.390364,-0.752973,0.246723,-0.776237,...,-0.869232,0.088524,-0.157855,-0.911090,-0.037018,-0.297428,-0.827384,0.325773,0.702706,-0.501927
A1CF,0.130143,0.562743,0.679178,-0.125639,0.088232,0.879337,-0.683533,-0.883483,-0.139563,0.581384,...,0.930605,-0.585922,-0.255846,0.832803,-0.395305,-0.037308,0.055690,-0.520815,-0.832303,-0.372069
A2M,0.400889,0.233444,0.321800,-0.278102,-0.752299,0.600927,-0.756958,0.857039,0.019484,0.307895,...,-0.891800,0.368335,0.810495,-0.329304,0.573028,0.428820,-0.003755,-0.217698,-0.961484,0.982789
A2ML1,-0.033140,-0.130751,-0.335376,0.715918,0.176146,-0.214502,0.124960,-0.163299,-0.502701,0.864906,...,0.590184,0.674018,0.799681,0.101691,-0.893250,0.627441,-0.628245,0.069148,0.259884,-0.228452
A3GALT2,-0.813436,0.977431,0.344382,0.162926,-0.715790,0.995998,0.595580,-0.543804,0.102436,-0.883197,...,0.367636,0.702692,-0.078872,0.204843,0.014125,-0.027690,-0.808785,0.511832,0.125751,0.725960


# Plot of A Single Celltype, Normalized Value vs. Standardized Value

In [72]:
plt.plot(normalized_matrix[normalized_matrix.columns[0]], standard_matrix[standard_matrix.columns[0]], 'bo')
plt.xlabel('Normalized Values')
plt.ylabel('Standardized Values')
plt.title(standard_matrix.columns[0])
plt.grid(True)

# Save Standardized Matrix

In [73]:
filename = path+'tcga_blca_standard_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
standard_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Tertiary Matrix

In [74]:
tertiary_matrix = uf.createTertiaryMatrix(standard_matrix)

In [75]:
tertiary_matrix.head()

,TCGA-2F-A9KO-01A,TCGA-2F-A9KP-01A,TCGA-2F-A9KQ-01A,TCGA-2F-A9KR-01A,TCGA-2F-A9KT-01A,TCGA-2F-A9KW-01A,TCGA-4Z-AA7M-01A,TCGA-4Z-AA7N-01A,TCGA-4Z-AA7O-01A,TCGA-4Z-AA7Q-01A,...,TCGA-ZF-AA4X-01A,TCGA-ZF-AA51-01A,TCGA-ZF-AA52-01A,TCGA-ZF-AA53-01A,TCGA-ZF-AA54-01A,TCGA-ZF-AA56-01A,TCGA-ZF-AA58-01A,TCGA-ZF-AA5H-01A,TCGA-ZF-AA5N-01A,TCGA-ZF-AA5P-01A
,,,,,,,,,,,,,,,,,,,,,
A1BG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,1
A2ML1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A3GALT2,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Save Teriary Matrix

In [76]:
filename = path+'tcga_blca_tertiary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
tertiary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Up Gene Set Library

In [77]:
name = 'tcga_blca_gene_up_set'

In [78]:
uf.createUpGeneSetLib(tertiary_matrix, path, name)

# Create Down Gene Set Library

In [79]:
name = 'tcga_blca_gene_down_set'

In [80]:
uf.createDownGeneSetLib(tertiary_matrix, path, name)

# Create Up Attribute Library

In [81]:
name = 'tcga_blca_attribute_up_set'

In [82]:
uf.createUpAttributeSetLib(tertiary_matrix, path, name)

# Create Down Attribute Library

In [83]:
name = 'tcga_blca_attribute_down_set'

In [84]:
uf.createDownAttributeSetLib(tertiary_matrix, path, name)

# Create Attribute Similarity matrix

In [85]:
attribute_normalized_matrix = normalized_matrix.copy()

In [86]:
getSampleData(attribute_normalized_matrix, sample_meta)

In [87]:
attribute_similarity_matix = uf.createSimilarityMatrix(attribute_normalized_matrix.T, 'cosine')

In [88]:
attribute_similarity_matix.head()

,"(Donor:TCGA-2F-A9KO, Age:63.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-2F-A9KP, Age:66.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-2F-A9KQ, Age:69.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-2F-A9KR, Age:59.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-2F-A9KT, Age:83.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-2F-A9KW, Age:67.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-4Z-AA7M, Age:65.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-4Z-AA7N, Age:65.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-4Z-AA7O, Age:64.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-4Z-AA7Q, Age:79.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)",...,"(Donor:TCGA-ZF-AA4X, Age:56.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-ZF-AA51, Age:69.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-ZF-AA52, Age:70.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-ZF-AA53, Age:60.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-ZF-AA54, Age:71.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-ZF-AA56, Age:79.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-ZF-AA58, Age:61.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-ZF-AA5H, Age:60.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-ZF-AA5N, Age:62.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Muscle invasive urothelial carcinoma (pT2 or above), Tissue:Bladder, Tumor:nan, GradeHigh Grade)","(Donor:TCGA-ZF-AA5P, Age:65.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Muscle invasive urothelial carcinoma (pT2 or above

# HeatMap (clustergrammer) of Similarity Matrix

In [92]:
# net.load_df(attribute_similarity_matix.iloc[:,:].copy())
# # net.filter_N_top('row', rank_type='sum', N_top=300)
# net.cluster()
# net.widget()

# Save Attribute Similarity Matrix

In [93]:
filename = path+'tcga_blca_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene Similarity Matrix

In [94]:
gene_similarity_matix = uf.createSimilarityMatrix(normalized_matrix, 'cosine')

In [95]:
gene_similarity_matix.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
,,,,,,,,,,,,,,,,,,,,,
A1BG,1.000000,-0.034179,-0.124040,-0.060425,0.013053,0.058576,-0.035842,0.108974,-0.066475,0.022011,...,0.038036,0.054225,-0.062083,-0.053878,-0.051192,-0.126218,-0.062466,-0.037119,0.001037,-0.056523
A1CF,-0.034179,1.000000,-0.107878,-0.029772,0.103633,-0.225372,0.093165,0.006349,-0.083384,0.053412,...,0.074657,0.011650,0.122014,0.109434,0.080547,0.107423,0.257082,-0.286728,0.054843,0.032593
A2M,-0.124040,-0.107878,1.000000,-0.153917,-0.202816,0.215029,-0.141156,-0.254656,-0.262455,-0.068925,...,-0.362981,-0.370900,-0.071485,-0.059360,-0.279993,-0.201755,0.138693,0.431809,0.092878,-0.066636
A2ML1,-0.060425,-0.029772,-0.153917,1.000000,-0.085622,0.098442,-0.006874,-0.150037,0.176909,0.083974,...,0.229321,0.229086,0.132128,-0.004466,0.221826,0.099840,-0.004373,-0.116248,-0.048922,0.168320
A3GALT2,0.013053,0.103633,-0.202816,-0.085622,1.000000,-0.006712,0.020076,0.149576,0.010891,0.002205,...,0.045487,0.112859,-0.059969,-0.063016,-0.005295,-0.040450,-0.036794,-0.097497,0.049004,-0.026974


# Save Gene Similarity Matrix

In [96]:
filename = path+'tcga_blca_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene-Attribute Edge List

In [97]:
name = 'tcga_blca_gene_attribute_edge_list'

In [98]:
uf.createGeneAttributeEdgeList(standard_matrix, attribute_list, gene_list, path, name)

Progeres: 100%  430 Out of 430   

 The number of statisticaly relevent gene-attribute associations is: 398197
